In [62]:
import requests
from bs4 import BeautifulSoup
import re
import json
from collections import defaultdict 
import time
from pandas import DataFrame
from geopy.geocoders import Nominatim
from statistics import mean
import numpy as np

In [2]:
class NextDoorInfo:
    def __init__(self):
        self.base_url = 'https://nextdoor.com/find-neighborhood/ca/'
        self.all_city_url = []
        self.all_neighborhood_url = []
        self.city_name_pattern = "(?<=city: \").*(?=\",)"
        self.neighbor_name_pattern = "(?<=shortName: \").*(?=\",)"
        self.attributes_pattern = "(?<=attributes: ).*(?=,)"
        self.coordinate_pattern = "(?<=mapBounds: ').*(?=',)"
        self.final_result = []
        self.cursor = 0
        
    def get_base_page(self):
        self.page = requests.get(self.base_url)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        if self.soup != None:
            print("Got Page")
    
    def get_all_city(self):
        self.results = self.soup.find(id = "child_links")
        self.job_elems = self.results.find_all(class_='link')
        pattern = "(?<=city\/).*(?=\-\-ca)"
        for i in self.job_elems:
            self.all_city_url.append(i['href'])
        print("First 5 Cities")
        for n in range(5):
            print (self.all_city_url[n])

    def get_all_neighborhood(self):
        for city_url in self.all_city_url:
            print(city_url)
            temp_url = []
            
            try:
                city_page = requests.get(city_url)
                city_soup = BeautifulSoup(city_page.content, 'html.parser')
                city_results = city_soup.find(id = 'child_links')
                city_job_elems = city_results.find_all(class_='link')

                for i in city_job_elems:
                    temp_url.append(i['href'])
                    
            except Exception:
                print("No Neighbor")
                
            self.all_neighborhood_url.append(temp_url)
            
    def get_all_tags(self):
        for place in range(self.cursor,len(self.all_neighborhood_url)):
            result = []
            time.sleep(1)
            if self.all_neighborhood_url[place] != 0:
                for neighbor in self.all_neighborhood_url[place]:
                    sub_result = []
                    print(neighbor)
                    neighborhood_page = requests.get(neighbor)
                    neighborhood_soup = BeautifulSoup(neighborhood_page.content, 'html.parser')
                    try:
                        city_name = re.search(self.city_name_pattern,str(neighborhood_soup)).group(0)
                        neighbor_name = re.search(self.neighbor_name_pattern,str(neighborhood_soup)).group(0)
                        attributes = json.loads(re.search(self.attributes_pattern,str(neighborhood_soup)).group(0))
                        coordinate = (re.search(self.coordinate_pattern,str(neighborhood_soup)).group(0)).split(',')
                        sub_result.append(city_name)
                        sub_result.append(self.all_city_url[place])
                        sub_result.append(neighbor_name)
                        sub_result.append(neighbor)
                        sub_result.append(attributes)
                        sub_result.append(mean([float(coordinate[0]),float(coordinate[2])]))
                        sub_result.append(mean([float(coordinate[1]),float(coordinate[3])]))
                    except AttributeError:
                        sub_result.append("Problem City")
                        sub_result.append(self.all_city_url[place])
                        sub_result.append("Problem Neighbor")
                        sub_result.append(neighbor)
                        sub_result.append("Problem Attributes")
                        sub_result.append("Problem Latitude")
                        sub_result.append("Problem Longitude")
                    result.append(sub_result)
            else:
                sub_result = []
                sub_result.append("City Name")
                sub_result.append(self.all_city_url[place])
                sub_result.append(None)
                sub_result.append(None)
                sub_result.append(None)
                sub_result.append(None)
                sub_result.append(None)
                result.append(sub_result)
        
            self.final_result.extend(result)
            self.cursor += 1
        
    def reset_cursor_n_results(self):
        self.cursor = 0
        self.final_result = []
        

Script

In [3]:
test = NextDoorInfo()
test.get_base_page()
test.get_all_city()
print(len(test.all_city_url))

Got Page
First 5 Cities
https://nextdoor.com/city/acampo--ca/
https://nextdoor.com/city/acton--ca/
https://nextdoor.com/city/adelaide--ca/
https://nextdoor.com/city/adelanto--ca/
https://nextdoor.com/city/adin--ca/
1164


In [4]:
test.get_all_neighborhood()
print(len(test.all_neighborhood_url))

https://nextdoor.com/city/acampo--ca/
https://nextdoor.com/city/acton--ca/
https://nextdoor.com/city/adelaide--ca/
No Neighbor
https://nextdoor.com/city/adelanto--ca/
https://nextdoor.com/city/adin--ca/
No Neighbor
https://nextdoor.com/city/agoura-hills--ca/
https://nextdoor.com/city/agua-dulce--ca/
https://nextdoor.com/city/aguanga--ca/
https://nextdoor.com/city/ahwahnee--ca/
https://nextdoor.com/city/alameda--ca/
https://nextdoor.com/city/alamo--ca/
https://nextdoor.com/city/albany--ca/
https://nextdoor.com/city/albion--ca/
https://nextdoor.com/city/alderpoint--ca/
No Neighbor
https://nextdoor.com/city/alhambra--ca/
https://nextdoor.com/city/aliso-viejo--ca/
https://nextdoor.com/city/alleghany--ca/
No Neighbor
https://nextdoor.com/city/alpaugh--ca/
No Neighbor
https://nextdoor.com/city/alpine--ca/
https://nextdoor.com/city/alpine-meadows--ca/
https://nextdoor.com/city/alta--ca/
https://nextdoor.com/city/altadena--ca/
https://nextdoor.com/city/altaville--ca/
No Neighbor
https://nextdo

https://nextdoor.com/city/ceres--ca/
https://nextdoor.com/city/cerritos--ca/
https://nextdoor.com/city/challenge--ca/
https://nextdoor.com/city/chatsworth--ca/
https://nextdoor.com/city/cherry-valley--ca/
https://nextdoor.com/city/chico--ca/
https://nextdoor.com/city/chinese-camp--ca/
No Neighbor
https://nextdoor.com/city/chino--ca/
https://nextdoor.com/city/chino-hills--ca/
https://nextdoor.com/city/chowchilla--ca/
https://nextdoor.com/city/chualar--ca/
https://nextdoor.com/city/chula-vista--ca/
https://nextdoor.com/city/citrus-heights--ca/
https://nextdoor.com/city/city-of-industry--ca/
https://nextdoor.com/city/claremont--ca/
https://nextdoor.com/city/clarksburg--ca/
https://nextdoor.com/city/clayton--ca/
https://nextdoor.com/city/clearlake--ca/
https://nextdoor.com/city/clearlake-oaks--ca/
https://nextdoor.com/city/cloverdale--ca/
https://nextdoor.com/city/clovis--ca/
https://nextdoor.com/city/coachella--ca/
https://nextdoor.com/city/coalinga--ca/
https://nextdoor.com/city/coarsego

https://nextdoor.com/city/geyserville--ca/
https://nextdoor.com/city/gilroy--ca/
https://nextdoor.com/city/glencoe--ca/
https://nextdoor.com/city/glendale--ca/
https://nextdoor.com/city/glendora--ca/
https://nextdoor.com/city/glen-ellen--ca/
https://nextdoor.com/city/glenhaven--ca/
https://nextdoor.com/city/glenn--ca/
No Neighbor
https://nextdoor.com/city/glennville--ca/
https://nextdoor.com/city/gold-river--ca/
https://nextdoor.com/city/gold-run--ca/
https://nextdoor.com/city/goleta--ca/
https://nextdoor.com/city/gonzales--ca/
https://nextdoor.com/city/goodyears-bar--ca/
No Neighbor
https://nextdoor.com/city/goshen--ca/
No Neighbor
https://nextdoor.com/city/granada-hills--ca/
https://nextdoor.com/city/grand-terrace--ca/
https://nextdoor.com/city/granite-bay--ca/
https://nextdoor.com/city/grass-valley--ca/
https://nextdoor.com/city/graton--ca/
https://nextdoor.com/city/greenbrae--ca/
https://nextdoor.com/city/greenfield--ca/
https://nextdoor.com/city/greenwood--ca/
https://nextdoor.com

https://nextdoor.com/city/lockwood--ca/
No Neighbor
https://nextdoor.com/city/lodi--ca/
https://nextdoor.com/city/loleta--ca/
https://nextdoor.com/city/loma-linda--ca/
https://nextdoor.com/city/loma-mar--ca/
https://nextdoor.com/city/lomita--ca/
https://nextdoor.com/city/lompoc--ca/
https://nextdoor.com/city/lone-pine--ca/
https://nextdoor.com/city/long-barn--ca/
https://nextdoor.com/city/long-beach--ca/
https://nextdoor.com/city/lookout--ca/
No Neighbor
https://nextdoor.com/city/loomis--ca/
https://nextdoor.com/city/los-alamitos--ca/
https://nextdoor.com/city/los-altos--ca/
https://nextdoor.com/city/los-altos-hills--ca/
https://nextdoor.com/city/los-angeles--ca/
https://nextdoor.com/city/los-banos--ca/
https://nextdoor.com/city/los-gatos--ca/
https://nextdoor.com/city/los-molinos--ca/
https://nextdoor.com/city/los-olivos--ca/
No Neighbor
https://nextdoor.com/city/los-osos--ca/
https://nextdoor.com/city/lost-hills--ca/
No Neighbor
https://nextdoor.com/city/lotus--ca/
No Neighbor
https:

https://nextdoor.com/city/palm-springs--ca/
https://nextdoor.com/city/palo-alto--ca/
https://nextdoor.com/city/palo-cedro--ca/
https://nextdoor.com/city/palomar-park--ca/
No Neighbor
https://nextdoor.com/city/palos-verdes-estates--ca/
https://nextdoor.com/city/palos-verdes-peninsula--ca/
No Neighbor
https://nextdoor.com/city/palo-verde--ca/
No Neighbor
https://nextdoor.com/city/panorama-city--ca/
https://nextdoor.com/city/paradise--ca/
https://nextdoor.com/city/paramount--ca/
https://nextdoor.com/city/parker-dam--ca/
No Neighbor
https://nextdoor.com/city/parlier--ca/
https://nextdoor.com/city/pasadena--ca/
https://nextdoor.com/city/paso-robles--ca/
https://nextdoor.com/city/patterson--ca/
https://nextdoor.com/city/pauma-valley--ca/
https://nextdoor.com/city/paynes-creek--ca/
No Neighbor
https://nextdoor.com/city/pearblossom--ca/
https://nextdoor.com/city/pebble-beach--ca/
https://nextdoor.com/city/penngrove--ca/
https://nextdoor.com/city/penn-valley--ca/
https://nextdoor.com/city/penry

No Neighbor
https://nextdoor.com/city/sausalito--ca/
https://nextdoor.com/city/scotia--ca/
No Neighbor
https://nextdoor.com/city/scott-bar--ca/
No Neighbor
https://nextdoor.com/city/scotts-valley--ca/
https://nextdoor.com/city/seal-beach--ca/
https://nextdoor.com/city/seaside--ca/
https://nextdoor.com/city/sebastopol--ca/
https://nextdoor.com/city/seeley--ca/
No Neighbor
https://nextdoor.com/city/seiad-valley--ca/
No Neighbor
https://nextdoor.com/city/selma--ca/
https://nextdoor.com/city/sequoia-national-park--ca/
No Neighbor
https://nextdoor.com/city/shafter--ca/
https://nextdoor.com/city/shandon--ca/
https://nextdoor.com/city/shasta--ca/
No Neighbor
https://nextdoor.com/city/shasta-lake--ca/
https://nextdoor.com/city/shaver-lake--ca/
https://nextdoor.com/city/sheep-ranch--ca/
https://nextdoor.com/city/shell-beach--ca/
No Neighbor
https://nextdoor.com/city/sheridan--ca/
https://nextdoor.com/city/sherman-oaks--ca/
https://nextdoor.com/city/shingle-springs--ca/
https://nextdoor.com/city

https://nextdoor.com/city/winchester--ca/
https://nextdoor.com/city/windsor--ca/
https://nextdoor.com/city/winnetka--ca/
https://nextdoor.com/city/winterhaven--ca/
No Neighbor
https://nextdoor.com/city/winters--ca/
https://nextdoor.com/city/winton--ca/
https://nextdoor.com/city/wishon--ca/
https://nextdoor.com/city/witter-springs--ca/
https://nextdoor.com/city/wofford-heights--ca/
https://nextdoor.com/city/woodacre--ca/
https://nextdoor.com/city/woodbridge--ca/
https://nextdoor.com/city/woodlake--ca/
https://nextdoor.com/city/woodland--ca/
https://nextdoor.com/city/woodland-hills--ca/
https://nextdoor.com/city/woodside--ca/
https://nextdoor.com/city/woodville--ca/
No Neighbor
https://nextdoor.com/city/woody--ca/
No Neighbor
https://nextdoor.com/city/wrightwood--ca/
https://nextdoor.com/city/yorba-linda--ca/
https://nextdoor.com/city/yorkville--ca/
https://nextdoor.com/city/yosemite-national-park--ca/
No Neighbor
https://nextdoor.com/city/yountville--ca/
https://nextdoor.com/city/yreka-

In [5]:
while test.cursor != len(test.all_city_url):
    try:
        test.get_all_tags()
    except:
        print("restarting")
        time.sleep(300)
print("done")

https://nextdoor.com/neighborhood/acampoacampo--acampo--ca/
https://nextdoor.com/neighborhood/collierrd--acampo--ca/
https://nextdoor.com/neighborhood/colliervillewest--acampo--ca/
https://nextdoor.com/neighborhood/dogtownacampo--acampo--ca/
https://nextdoor.com/neighborhood/dustinrd--acampo--ca/
https://nextdoor.com/neighborhood/forestlakecommunity--acampo--ca/
https://nextdoor.com/neighborhood/grahamrdca--acampo--ca/
https://nextdoor.com/neighborhood/woodbridgeand99--acampo--ca/
https://nextdoor.com/neighborhood/centralacton--acton--ca/
https://nextdoor.com/neighborhood/shannonvalleyca--acton--ca/
https://nextdoor.com/neighborhood/adelantoca--adelanto--ca/
https://nextdoor.com/neighborhood/americantraditions--adelanto--ca/
https://nextdoor.com/neighborhood/asterrdca--adelanto--ca/
https://nextdoor.com/neighborhood/chestnutca--adelanto--ca/
https://nextdoor.com/neighborhood/dospalmasmaricopa--adelanto--ca/
https://nextdoor.com/neighborhood/expressway--adelanto--ca/
https://nextdoor.co

https://nextdoor.com/neighborhood/applause--aliso-viejo--ca/
https://nextdoor.com/neighborhood/avanimalhospital--aliso-viejo--ca/
https://nextdoor.com/neighborhood/californiareflections--aliso-viejo--ca/
https://nextdoor.com/neighborhood/californiasummit--aliso-viejo--ca/
https://nextdoor.com/neighborhood/camdenpark--aliso-viejo--ca/
https://nextdoor.com/neighborhood/cantora--aliso-viejo--ca/
https://nextdoor.com/neighborhood/canyonvillas--aliso-viejo--ca/
https://nextdoor.com/neighborhood/capevictoria--aliso-viejo--ca/
https://nextdoor.com/neighborhood/glenwood--aliso-viejo--ca/
https://nextdoor.com/neighborhood/heatherridgeav--aliso-viejo--ca/
https://nextdoor.com/neighborhood/lagunaaudubon--aliso-viejo--ca/
https://nextdoor.com/neighborhood/leonsca--aliso-viejo--ca/
https://nextdoor.com/neighborhood/montelenaalisoviejo--aliso-viejo--ca/
https://nextdoor.com/neighborhood/morningside--aliso-viejo--ca/
https://nextdoor.com/neighborhood/moultonpkwy--aliso-viejo--ca/
https://nextdoor.com

https://nextdoor.com/neighborhood/orangewoodanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/palmlaneanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/paulrevereanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/pelanconiparkanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/peraltahillsanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/pioneerparkanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/platinumtriangleanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/ponderosanorthanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/rancholapalmaanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/riovistaanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/romneyaanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/ronaldreaganparkanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/rossparkanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/runningspringsanaheim--anaheim--ca/
https://nextdoor.com/neighborhood/sageparkanaheim--anahe

https://nextdoor.com/neighborhood/contralomaestates--antioch--ca/
https://nextdoor.com/neighborhood/countrymanorca--antioch--ca/
https://nextdoor.com/neighborhood/cstreetca--antioch--ca/
https://nextdoor.com/neighborhood/dallasranchestates--antioch--ca/
https://nextdoor.com/neighborhood/daybreakca--antioch--ca/
https://nextdoor.com/neighborhood/deerfieldparkca--antioch--ca/
https://nextdoor.com/neighborhood/deerridgefieldcrest--antioch--ca/
https://nextdoor.com/neighborhood/deerspringsungrove--antioch--ca/
https://nextdoor.com/neighborhood/deervalleyantioch--antioch--ca/
https://nextdoor.com/neighborhood/eagleridgeca--antioch--ca/
https://nextdoor.com/neighborhood/etregallasgarrowdavison--antioch--ca/
https://nextdoor.com/neighborhood/fingerpeakcommunity--antioch--ca/
https://nextdoor.com/neighborhood/fosterhills--antioch--ca/
https://nextdoor.com/neighborhood/gardenterraceca--antioch--ca/
https://nextdoor.com/neighborhood/garrowestates--antioch--ca/
https://nextdoor.com/neighborhood/g

https://nextdoor.com/neighborhood/sycamoreapplevalley--apple-valley--ca/
https://nextdoor.com/neighborhood/tawya--apple-valley--ca/
https://nextdoor.com/neighborhood/theestatesapplevalley--apple-valley--ca/
https://nextdoor.com/neighborhood/thevillageapplevalley--apple-valley--ca/
https://nextdoor.com/neighborhood/tomahawkca--apple-valley--ca/
https://nextdoor.com/neighborhood/vistadelsolestates--apple-valley--ca/
https://nextdoor.com/neighborhood/yakimaalgonquin--apple-valley--ca/
https://nextdoor.com/neighborhood/aptos--aptos--ca/
https://nextdoor.com/neighborhood/cabrillocollege--aptos--ca/
https://nextdoor.com/neighborhood/dayvalley--aptos--ca/
https://nextdoor.com/neighborhood/loyolalosaltosloop--aptos--ca/
https://nextdoor.com/neighborhood/marvistaca--aptos--ca/
https://nextdoor.com/neighborhood/nisenemarks--aptos--ca/
https://nextdoor.com/neighborhood/pebblebeachpinehurst--aptos--ca/
https://nextdoor.com/neighborhood/pleasantvalleyscc--aptos--ca/
https://nextdoor.com/neighborhoo

...

https://nextdoor.com/neighborhood/rbw--walnut-creek--ca/
https://nextdoor.com/neighborhood/rossmoorca--walnut-creek--ca/
https://nextdoor.com/neighborhood/rudgearestates--walnut-creek--ca/
https://nextdoor.com/neighborhood/rudgearmeadows--walnut-creek--ca/
https://nextdoor.com/neighborhood/sanmarcoca--walnut-creek--ca/
https://nextdoor.com/neighborhood/sanmiguelwalnut--walnut-creek--ca/
https://nextdoor.com/neighborhood/saranap--walnut-creek--ca/
https://nextdoor.com/neighborhood/skywestca--walnut-creek--ca/
https://nextdoor.com/neighborhood/sorrel--walnut-creek--ca/
https://nextdoor.com/neighborhood/summitridge--walnut-creek--ca/
https://nextdoor.com/neighborhood/ticevalley--walnut-creek--ca/
https://nextdoor.com/neighborhood/waldenparkca--walnut-creek--ca/
https://nextdoor.com/neighborhood/waldon--walnut-creek--ca/
https://nextdoor.com/neighborhood/walnutcreekca--walnut-creek--ca/
https://nextdoor.com/neighborhood/walnutheightswalnutcreek--walnut-creek--ca/
https://nextdoor.com/neigh

https://nextdoor.com/neighborhood/solanapark--west-covina--ca/
https://nextdoor.com/neighborhood/southhillsca--west-covina--ca/
https://nextdoor.com/neighborhood/thesecret--west-covina--ca/
https://nextdoor.com/neighborhood/traweekwestcovina--west-covina--ca/
https://nextdoor.com/neighborhood/vincentave--west-covina--ca/
https://nextdoor.com/neighborhood/walnutcreekparkway--west-covina--ca/
https://nextdoor.com/neighborhood/wescovepl--west-covina--ca/
https://nextdoor.com/neighborhood/westcovinanw--west-covina--ca/
https://nextdoor.com/neighborhood/woodsidevillageca--west-covina--ca/
https://nextdoor.com/neighborhood/asmancapistranosaleave--west-hills--ca/
https://nextdoor.com/neighborhood/balmorallane--west-hills--ca/
https://nextdoor.com/neighborhood/castlepeakestates--west-hills--ca/
https://nextdoor.com/neighborhood/chaminadeca--west-hills--ca/
https://nextdoor.com/neighborhood/fallbrookavenuecommunity--west-hills--ca/
https://nextdoor.com/neighborhood/fallbrookmall--west-hills--ca

https://nextdoor.com/neighborhood/westcapitolave--west-sacramento--ca/
https://nextdoor.com/neighborhood/westsacramento--west-sacramento--ca/
https://nextdoor.com/neighborhood/westsidewestsacramento--west-sacramento--ca/
https://nextdoor.com/neighborhood/hamiltonbranch--westwood--ca/
https://nextdoor.com/neighborhood/lakealmanor--westwood--ca/
https://nextdoor.com/neighborhood/westwoodca1--westwood--ca/
https://nextdoor.com/neighborhood/campfarwest--wheatland--ca/
https://nextdoor.com/neighborhood/wheatland--wheatland--ca/
https://nextdoor.com/neighborhood/sheltercoveca--whitethorn--ca/
https://nextdoor.com/neighborhood/62pierson--whitewater--ca/
https://nextdoor.com/neighborhood/paintedhillsca--whitewater--ca/
https://nextdoor.com/neighborhood/sagebrushave--whitewater--ca/
https://nextdoor.com/neighborhood/skyborne--whitewater--ca/
https://nextdoor.com/neighborhood/snowcreekca--whitewater--ca/
https://nextdoor.com/neighborhood/whitewaternorth--whitewater--ca/
https://nextdoor.com/neig

https://nextdoor.com/neighborhood/frigateca--wilmington--ca/
https://nextdoor.com/neighborhood/harborpinecreek--wilmington--ca/
https://nextdoor.com/neighborhood/lomitato101--wilmington--ca/
https://nextdoor.com/neighborhood/wilmingtonneptune--wilmington--ca/
https://nextdoor.com/neighborhood/northwestwilmington--wilmington--ca/
https://nextdoor.com/neighborhood/wdenniwanaheim--wilmington--ca/
https://nextdoor.com/neighborhood/westmst--wilmington--ca/
https://nextdoor.com/neighborhood/wlst--wilmington--ca/
https://nextdoor.com/neighborhood/wiltonca--wilton--ca/
https://nextdoor.com/neighborhood/frenchvalley--winchester--ca/
https://nextdoor.com/neighborhood/fvspencerscrossing--winchester--ca/
https://nextdoor.com/neighborhood/panoramacove--winchester--ca/
https://nextdoor.com/neighborhood/pattersonca--winchester--ca/
https://nextdoor.com/neighborhood/simpsonrd--winchester--ca/
https://nextdoor.com/neighborhood/stetsonca--winchester--ca/
https://nextdoor.com/neighborhood/winchesterwinch

https://nextdoor.com/neighborhood/brokenhillctca--yorba-linda--ca/
https://nextdoor.com/neighborhood/charterhillprominence--yorba-linda--ca/
https://nextdoor.com/neighborhood/clubviewdrca--yorba-linda--ca/
https://nextdoor.com/neighborhood/countryestatesyorbalinda--yorba-linda--ca/
https://nextdoor.com/neighborhood/countryhomes--yorba-linda--ca/
https://nextdoor.com/neighborhood/eastlakecommunity--yorba-linda--ca/
https://nextdoor.com/neighborhood/eastlakeheights--yorba-linda--ca/
https://nextdoor.com/neighborhood/eastlaketravis--yorba-linda--ca/
https://nextdoor.com/neighborhood/eastlakevillage--yorba-linda--ca/
https://nextdoor.com/neighborhood/esparanza--yorba-linda--ca/
https://nextdoor.com/neighborhood/fairgreenca--yorba-linda--ca/
https://nextdoor.com/neighborhood/fairmonthillcondoassociation--yorba-linda--ca/
https://nextdoor.com/neighborhood/fairmontknolls--yorba-linda--ca/
https://nextdoor.com/neighborhood/fernglentriangle--yorba-linda--ca/
https://nextdoor.com/neighborhood/fo

In [6]:
test.final_result

[['Acampo',
  'https://nextdoor.com/city/acampo--ca/',
  'Acampo',
  'https://nextdoor.com/neighborhood/acampoacampo--acampo--ca/',
  ['Quiet',
   'Rural',
   'Safe',
   'Dog Friendly',
   'Family Friendly',
   'Peaceful',
   'Lake',
   'Nature',
   'Neighbors',
   'Nice'],
  38.177751,
  -121.26072400000001],
 ['Acampo',
  'https://nextdoor.com/city/acampo--ca/',
  'Collier Rd',
  'https://nextdoor.com/neighborhood/collierrd--acampo--ca/',
  ['Safe',
   'Peaceful',
   'Quiet',
   'Freeway Access',
   'Open',
   'Privacy',
   'Rural',
   'Beautiful',
   'Nature',
   'Trees'],
  38.21849264679226,
  -121.24688325568388],
 ['Acampo',
  'https://nextdoor.com/city/acampo--ca/',
  'Collierville West',
  'https://nextdoor.com/neighborhood/colliervillewest--acampo--ca/',
  ['Peaceful',
   'Friendly',
   'Dog Friendly',
   'Open',
   'Privacy',
   'Quiet',
   'Rural',
   'Walking',
   'Atmosphere',
   'Beautiful'],
  38.21185006912226,
  -121.27958117820714],
 ['Acampo',
  'https://nextdoor.co

In [7]:
test.cursor

1164

In [9]:
df = DataFrame (test.final_result,columns=['city_name','city_url','neighborhood_name','neighborhood_url','tags','latitude','longitude'])

In [17]:
temp = df
temp = temp.astype({'latitude':str,'longitude':str})
temp['raw_address'] = temp['latitude'] + ','+ temp['longitude']

In [12]:
geolocator = Nominatim(user_agent="170Project")
temp['raw_address'] = temp['raw_address'].apply(geolocator.reverse)

In [58]:
pattern = "(?<=\,\s)9[0-9][0-9][0-9][0-9](?=\,)"
result_list = []
for i in temp['raw_address']:
    try:
        result_list.append(re.search(pattern,str(i)).group(0))
    except AttributeError:
        result_list.append(None)
temp['zip_code'] = result_list

In [67]:
df = temp

,city_name,city_url,neighborhood_name,neighborhood_url,tags,latitude,longitude,raw_address,zip_code
0,Acampo,https://nextdoor.com/city/acampo--ca/,Acampo,https://nextdoor.com/neighborhood/acampoacampo...,"[Quiet, Rural, Safe, Dog Friendly, Family Frie...",38.177751,-121.26072400000001,"(99 Frontage Road, San Joaquin County, Califor...",95220
1,Acampo,https://nextdoor.com/city/acampo--ca/,Collier Rd,https://nextdoor.com/neighborhood/collierrd--a...,"[Safe, Peaceful, Quiet, Freeway Access, Open, ...",38.21849264679226,-121.24688325568388,"(5704, Collier Road, Collierville, San Joaquin...",95220
2,Acampo,https://nextdoor.com/city/acampo--ca/,Collierville West,https://nextdoor.com/neighborhood/collierville...,"[Peaceful, Friendly, Dog Friendly, Open, Priva...",38.21185006912226,-121.27958117820714,"(3088, East Woodson Road, Collierville, San Jo...",95220
3,Acampo,https://nextdoor.com/city/acampo--ca/,Dogtown,https://nextdoor.com/neighborhood/dogtownacamp...,"[Peaceful, Quiet, Rural, Privacy, Family Frien...",38.21367847067797,-121.13870261062989,"(24943, Fuhrman Road, San Joaquin County, Cali...",95220
4,Acampo,https://nextdoor.com/city/acampo--ca/,Dustin Rd,https://nextdoor.com/neighborhood/dustinrd--ac...,None,38.1503665,-121.21890150000002,"(Bunie Lane, San Joaquin County, California, U...",None
...,...,...,...,...,...,...,...,...,...
17901,Yucca Valley,https://nextdoor.com/city/yucca-valley--ca/,Storey Park,https://nextdoor.com/neighborhood/storeypark--...,"[Peaceful, Quiet, Beautiful, Safe, Family Frie...",34.097479796352964,-116.4250218013444,"(56781, Golden Bee Drive, Yucca Valley, San Be...",92284
17902,Yucca Valley,https://nextdoor.com/city/yucca-valley--ca/,Sunland Dr,https://nextdoor.com/neighborhood/sunlanddr--y...,"[Beautiful, Dog Friendly, Nature, Neighbors, P...",34.128057999999996,-116.45587618383789,"(Shawnee Trail, Yucca Valley, San Bernardino C...",92284
17903,Yucca Valley,https://nextdoor.com/city/yucca-valley--ca/,Valley Vista Ave,https://nextdoor.com/neighborhood/valleyvistaa...,"[Atmosphere, Neighbors, Architecture, Beautifu...",34.109331,-116.41116349999999,"(57477, Navajo Trail, Yucca Valley, San Bernar...",92284
17904,Yucca Valley,https://nextdoor.com/city/yucca-valley--ca/,Western Hills,https://nextdoor.com/neighborhood/westernhills...,"[Peaceful, View, Quiet, Hills, Atmosphere, Nei...",34.1419845,-116.41343049999998,"(6021, Old Woman Springs Road, Yucca Valley, S...",92284


In [68]:
df.to_csv(r'F:\170A Data\all_neighbors_zip.csv',index = False, header = True)